In [1]:
#install necessaries
!pip install pytorchvideo
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for av from https://files.pythonhosted.org/packages/0a/32/56aaa677f0ec369eb68623a2ade90358c1a51b6e3cd6087111bd1e096b82/av-11.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for portalocker from https://files.pythonhosted.org/packages/17/9e/87671efcca80ba6203811540ed1f9c0462c1609d2281d7b7f53cef05da3d/portalocker-2.8.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 36.8 MB/s eta 0:00:00:00:0100:01
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188685 sha256=

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Process data

In [4]:
import cv2
import os
from glob import glob

In [ ]:
temporal = '/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Temporal_Anomaly_Annotation_for_Testing_Videos.txt'
data = pd.read_csv(temporal,sep='  ',header=None, names=['name','events','s1','e1','s2','e2'])
data

In [ ]:
def transformFrame(frame, size):
    frame = cv2.resize(frame, size, interpolation=cv2.INTER_AREA)
    return frame
def hflip(frame):
    return cv2.flip(frame, 1)
def noisy(img, noise_type="gauss"):
    '''
    ### Adding Noise ###
    img: image
    cj_type: {gauss: gaussian, sp: salt & pepper}
    '''
    if noise_type == "gauss":
        image=img.copy() 
        mean=0
        st=0.5
        gauss = np.random.normal(mean,st,image.shape)
        gauss = gauss.astype('uint8')
        image = cv2.add(image,gauss)
        return image
    
    elif noise_type == "sp":
        image=img.copy() 
        prob = 0.005
        if len(image.shape) == 2:
            black = 0
            white = 255            
        else:
            colorspace = image.shape[2]
            if colorspace == 3:  # RGB
                black = np.array([0, 0, 0], dtype='uint8')
                white = np.array([255, 255, 255], dtype='uint8')
            else:  # RGBA
                black = np.array([0, 0, 0, 255], dtype='uint8')
                white = np.array([255, 255, 255, 255], dtype='uint8')
        probs = np.random.random(image.shape[:2])
        image[probs < (prob / 2)] = black
        image[probs > 1 - (prob / 2)] = white
        return image
    
    else:
        return img

In [ ]:
def saveVideo(path, out, start, end, num, size, **kwargs):
    vid = cv2.VideoCapture(path)
    length = vid.get(cv2.CAP_PROP_FRAME_COUNT)
    first = start
    if(end>0 and end<=length):
        step = ((end-start)/num)
    else:
        first=1
        step = min(length/2, 1024)/num
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    res = cv2.VideoWriter(out, fourcc, 10, size)
    for i in range(num):
        vid.set(cv2.CAP_PROP_POS_FRAMES, int(first + i*step)) # xem lai start tu 0 hay 1
        ret, frame = vid.read()
        frame = transformFrame(frame, size)
        if(kwargs.get('flip', False)):
            frame = hflip(frame)
        frame = noisy(frame, kwargs.get('noise', 'none'))
        res.write(frame)
    res.release()
    vid.release()
    
paths = {
    'Abuse':'/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/',
    'Arrest':'/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Arrest/',
    'Arson':'/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Arson/',
    'Assault':'/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Assault/',
    'Burglary':'/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2/Burglary/',
    'Explosion':'/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2/Explosion/',
    'Fighting':'/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2/Fighting/',
    'Normal':'/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1/'
} 

In [ ]:
# path = '/kaggle/working/test.mp4'
# saveVideo('/kaggle/input/crimeucfdataset/Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1/Abuse/Abuse001_x264.mp4',path,1,600,32,(32,32))

In [5]:
import os
from glob import glob

# normal = glob('/kaggle/working/abnormal/*')
# abnormal = glob('/kaggle/working/normal/*')
# for f in normal:
#     os.remove(f)
# for f in abnormal:
#     os.remove(f)
# os.makedirs('/kaggle/working/abnormal')
# os.makedirs('/kaggle/working/normal')

In [ ]:
size = (128, 128)
num = 128
ab = 0
nm = 0

for i in range(len(data)):
    name = '/kaggle/working/'
    flip = [True, False]
    noise = ['none', 'gauss']
    if data['events'][i] not in paths.keys():
        continue
    if i < 188:
        continue
    path = paths[data['events'][i]] + data['name'][i]
    if(data['events'][i] != 'Normal'):
        name += 'abnormal/'
        s1 = data['s1'][i]
        e1 = data['e1'][i]
        s2 = data['s2'][i]
        e2 = data['e2'][i]
        if(s1>0 and e1>0):
            for j in flip:
                for k in noise:
                    ab += 1
                    idx = 'ab_'+"{:03d}".format(ab)+'.mp4'
                    saveVideo(path, name+idx, s1, e1, num, size, flip=j, noise=k)
        if(s2>0 and e2>0):
            for j in flip:
                for k in noise:
                    ab += 1
                    idx = 'ab_'+"{:03d}".format(ab)+'.mp4'
                    saveVideo(path, name+idx, s1, e1, num, size, flip=j, noise=k)
    else:
        name += 'normal/'
        for k in noise:
            nm += 1
            idx = 'nm_'+"{:03d}".format(nm)+'.mp4'
            saveVideo(path, name+idx, 1, -1, num, size, flip=False, noise=k)
    print(str(i) + ' ' + data['name'][i])

## Dataloader

In [6]:
normal = glob('/kaggle/working/normal/*')
abnormal = glob('/kaggle/working/abnormal/*')
label = [0]*len(normal)+[1]*len(abnormal)
df = pd.DataFrame(zip(normal+abnormal, label), columns=['file', 'label'])
print('Normal: ', len(normal))
print('Abnormal: ', len(abnormal))
df['label'] = df['label'].astype(int)
df.head()

Normal:  300
Abnormal:  252


,file,label
0,/kaggle/working/normal/nm_071.mp4,0
1,/kaggle/working/normal/nm_037.mp4,0
2,/kaggle/working/normal/nm_078.mp4,0
3,/kaggle/working/normal/nm_158.mp4,0
4,/kaggle/working/normal/nm_252.mp4,0


In [7]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, shuffle=True)
len(train_df), len(val_df)
train_df

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,file,label
508,/kaggle/working/abnormal/ab_135.mp4,1
3,/kaggle/working/normal/nm_158.mp4,0
172,/kaggle/working/normal/nm_148.mp4,0
101,/kaggle/working/normal/nm_056.mp4,0
144,/kaggle/working/normal/nm_007.mp4,0
...,...,...
133,/kaggle/working/normal/nm_296.mp4,0
428,/kaggle/working/abnormal/ab_028.mp4,1
138,/kaggle/working/normal/nm_061.mp4,0
415,/kaggle/working/abnormal/ab_221.mp4,1


In [8]:
train_df.to_csv('train.txt', sep=' ', index=False, header=False)
val_df.to_csv('val.txt', sep=' ', index=False, header=False)

In [9]:
train_lst = list(train_df.itertuples(index=False, name=None))
val_lst = list(val_df.itertuples(index=False, name=None))
print(train_lst[0])

('/kaggle/working/abnormal/ab_135.mp4', 1)


In [10]:
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler, labeled_video_dataset
from pytorch_lightning import LightningDataModule

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    UniformTemporalSubsample,
    Permute
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize
)

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo
)

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [38]:
video_transform=Compose([
    ApplyTransformToKey(
        key='video',
        transform = Compose([
        UniformTemporalSubsample(64),
        Lambda(lambda x: x/255),
        Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
        RandomHorizontalFlip(p=0.5)
        ])
    ),
    ApplyTransformToKey(
        key='label',
        transform = Lambda(lambda x: torch.tensor([float(x)]))
    )
])

In [39]:
from torch.utils.data import DataLoader
class MyDataModule(LightningDataModule):
    def __init__(self):
        super().__init__()
        self.BATCH_SIZE = 4
        self.NUM_WORKERS = 2
    
    def train_dataloader(self):
        train_dataset = labeled_video_dataset(
            '/kaggle/working/train.txt',
            clip_sampler=make_clip_sampler('random', 12.8),
            transform=video_transform,
            decode_audio=False
        )
        train_loader = DataLoader(train_dataset, batch_size=self.BATCH_SIZE, num_workers=self.NUM_WORKERS, shuffle=False)
        return train_loader
    
    def val_dataloader(self):
        val_dataset = labeled_video_dataset(
            '/kaggle/working/val.txt',
            clip_sampler=make_clip_sampler('random', 12.8),
            transform=video_transform,
            decode_audio=False
        )
        val_loader = DataLoader(val_dataset, batch_size=self.BATCH_SIZE, num_workers=self.NUM_WORKERS, shuffle=False)
        return val_loader
    
    def test_dataloader(self):
        val_dataset = labeled_video_dataset(
            '/kaggle/working/val.txt',
            clip_sampler=make_clip_sampler('random', 6.4),
            transform=video_transform,
            decode_audio=False
        )
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)
        return val_loader

In [40]:
dm = MyDataModule()
batch=next(iter(dm.val_dataloader()))
batch.keys()

dict_keys(['video', 'video_name', 'video_index', 'clip_index', 'aug_index', 'label'])

In [42]:
batch['video'].shape, batch['label'].shape, batch['video_name'], batch['label']

(torch.Size([2, 3, 64, 128, 128]),
 torch.Size([2, 1]),
 ['nm_133.mp4', 'nm_058.mp4'],
 tensor([[0.],
         [0.]]))

## Model

In [43]:
import torch
import torch.nn as nn
from pytorch_lightning import LightningModule, seed_everything, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import classification_report
import torchmetrics

In [44]:
video_model = torch.hub.load('facebookresearch/pytorchvideo', model='efficient_x3d_xs', pretrained=True)

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [47]:
class MyModel(LightningModule):
    def __init__(self):
        super().__init__()
        vid_mod = torch.hub.load('facebookresearch/pytorchvideo', model='efficient_x3d_xs', pretrained=True)
        self.vid_model = vid_mod
        self.relu = nn.ReLU()
        self.linear = nn.Linear(400, 1)
        #parameters
        self.lr=1e-3
        self.batch_size = 4
        #loss function
        self.loss = nn.BCEWithLogitsLoss()
    
    def forward(self, x):
        x = self.vid_model(x)
        x = self.relu(x)
        x = self.linear(x)
        return x
        
    def training_step(self, batch, batch_idx):
        y_hat = self(batch['video'])
        loss = self.loss(y_hat, batch['label'])
        self.log("train_loss", loss, sync_dist=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        y_hat = self(batch['video'])
        loss = self.loss(y_hat, batch['label'])
        self.log("val_loss", loss, sync_dist=True)
        return loss
    
    def configure_optimizers(self):
        opt = torch.optim.AdamW(params=self.parameters(), lr=self.lr)
        return {'optimizer':opt}

In [48]:
m = MyModel()
#print(batch['video'])
out = m(batch['video'])
print(out.shape, out)
lab = batch['label']
print(lab.shape, lab)
l = m.loss
print(l(out, lab))

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


torch.Size([2, 1]) tensor([[-0.5269],
        [ 0.1899]], grad_fn=<AddmmBackward0>)
torch.Size([2, 1]) tensor([[0.],
        [0.]])
tensor(0.6283, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


## Training

In [49]:
from pytorch_lightning.loggers import WandbLogger
import wandb
wandb.login(key="1658d378d091cf8659e37004bc727f76b3de8356")
wandb.init(project="banhgao1")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: duongchinh29062003 (chinh). Use `wandb login --relogin` to force relogin


In [53]:
checkpoint_callback = ModelCheckpoint(monitor='val_loss',mode='min',save_top_k=10,dirpath='/kaggle/working/log',filename='banhgao-{val_loss:.2f}')
lr_monitor = LearningRateMonitor(logging_interval='epoch')
#os.makedirs("/kaggle/working/log")

In [54]:
model = MyModel()
data_module = MyDataModule()
wandb_logger = WandbLogger(log_model='all',save_dir="/kaggle/working/log",project='banhgao1')
trainer = Trainer(devices=2, accelerator='gpu', max_epochs=32, logger = wandb_logger, default_root_dir="/kaggle/working/log")
trainer.fit(model,data_module)
wandb.finish()

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [21]:
wandb.finish()